In [1]:
import numpy as np
import pandas as pd
import openGeoJson
allCoordinates, allPairsLen, allPairsMat = openGeoJson.loadGeoJson()
signals = pd.read_csv('signalisation.csv')

0  /  90982
10000  /  90982
20000  /  90982
30000  /  90982
40000  /  90982
50000  /  90982
60000  /  90982
70000  /  90982
80000  /  90982
90000  /  90982


In [101]:
np.save('allPairsMat', allPairsMat)

In [2]:
print(signals.columns.values)
print(signals.iloc[0])

['X' 'Y' 'POTEAU_ID_POT' 'PANNEAU_ID_PAN' 'PANNEAU_ID_RPA'
 'DESCRIPTION_RPA' 'CODE_RPA' 'FLECHE_PAN' 'TOPONYME_PAN'
 'POTEAU_VERSION_POT' 'DATE_CONCEPTION_POT' 'PAS_SUR_RUE' 'DESCRIPTION_REP'
 'DESCRIPTION_RTP' 'X_EPSG32188' 'Y_EPSG32188' 'Longitude' 'Latitude'
 'DESCRIPTION_CAT' 'NOM_ARR']
X                                                             -73.5668
Y                                                               45.576
POTEAU_ID_POT                                                   240002
PANNEAU_ID_PAN                                                 1188478
PANNEAU_ID_RPA                                                    1762
DESCRIPTION_RPA        \P RESERVE DEBARCADERE HANDICAPES EN TOUT TEMPS
CODE_RPA                                                         RD-TT
FLECHE_PAN                                                           2
TOPONYME_PAN                                                       NaN
POTEAU_VERSION_POT                                                  

In [3]:
signalCoordinates = np.matrix( [signals['Latitude'].values, signals['Longitude'].values] ).T
print(signalCoordinates.shape, signalCoordinates[0,:])

(313731, 2) [[ 45.57600384 -73.5668386 ]]


In [4]:
print(allPairsMat.shape, allPairsLen, allPairsMat[10,:])
print('number of signs: ', len(signals), 150000 *23.9 /1000 / 60 / 60)

(324917, 4) 96108 [[ 45.45900367 -73.89426517  45.4589526  -73.89422442]]
number of signs:  313731 0.9958333333333333


In [87]:
from numpy import matlib
def innerPd(pt0, pt1):
    return np.multiply(pt0[:,0],pt1[:,0]) + np.multiply(pt0[:,1],pt1[:,1])
def projPt(lnPt0, lnPt1, pt):
    pt = pt - lnPt0
    lnPt1 = lnPt1 - lnPt0
    lnLen = np.sqrt(innerPd(lnPt1, lnPt1))
    coefs3 = innerPd(pt, lnPt1)
    coefs = np.multiply( np.sign(innerPd(pt, lnPt1)) , np.sqrt( np.absolute(innerPd(pt, lnPt1)) ) )
    #coefs2 = np.multiply(( np.greater(coefs, lnLen) ) , lnLen) + np.multiply( ( np.logical_and( np.greater_equal(coefs, np.zeros(coefs.shape)) , np.less_equal(coefs, lnLen) ) ) , coefs)
    #coefs2 = np.multiply(( np.greater(coefs, lnLen) ) , lnLen) + np.multiply( ( np.logical_and( np.greater_equal(coefs, np.zeros(coefs.shape)) , np.less_equal(coefs, lnLen) ) ) , coefs)
    coefs2 = coefs
    ppp = np.multiply(matlib.repmat(coefs2, 1, 2) , lnPt1 / lnLen /2) 
    pjs = lnPt0 + ppp
    #print(lnPt0[6], '-', lnPt1[6])
    #print(lnLen[6], coefs[6], coefs2[6], ppp[6], coefs3[6])
    return pjs
def distLmtd(lnPt0, lnPt1, pt):
    pj = projPt(lnPt0, lnPt1, pt)
    dist = np.power( np.power(pt[:,0] - pj[:,0],2) + np.power(pt[:,1]-pj[:,1],2) ,0.5 )
    return (dist, pj)
def closeTo(allMat, sign):
    filter_ = np.ones( [allMat.shape[0],1] )
    idRules = [[0,0],[0,2],[1,1],[1,3]]
    for rule in idRules:
        #print(rule)
        filter_ = np.logical_and( filter_, np.less( np.absolute( allMat[:, rule[1]] - sign.item(rule[0]) ), 0.004 ) )
    #filter_ = matlib.repmat(filter_, 1, 4)
    filter_ = filter_.tolist()
    filter_ = np.array(filter_)
    filter_ = filter_.T
    filter_ = filter_[0]
    #print(filter_.shape, allMat.shape, filter_)
    return allMat[ filter_ , :]
def findClosest(allPairsMat_, sign):
    allPairsMat_ = closeTo(allPairsMat_, sign)
    #print(allPairsMat_.shape)
    dists, pjs = distLmtd( allPairsMat_[:,0:2], allPairsMat_[:,2:4], sign )
    minIdx = np.argmin(dists)
    return ( dists[minIdx], allPairsMat_[minIdx, :], pjs[minIdx], pjs, dists, allPairsMat_ )

In [88]:
import folium
import matplotlib.pyplot as plt
%matplotlib inline
map_ = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
idx  = 53901 #251
dist, pair, pj, pjs, dists, allMats = findClosest( allPairsMat, signalCoordinates[idx,:]  )

#sortIds = np.argsort(dists, axis = 0)
#sortIds = sortIds[0:100]
#sortIds = sortIds[20]

colors = ['red', 'green', 'black', 'darkred', 'gray']

for id_index, id_ in enumerate(sortIds):
    if id_index != 6:
        continue
    idd = id_.item(0)
    pj_ = pjs[idd]
    c = colors[np.random.randint(1, len(colors), 1)]
    print(c)
    folium.Marker(pj_.tolist()[0], popup = str(dists[idd]) + ', ' + str(idd) , icon = folium.Icon(color =c)).add_to(map_)
    line = allMats[idd,:].tolist()[0]
    line = [(line[0], line[1]), (line[2], line[3])]

    folium.Marker(line[0], popup = 'pt0' , icon = folium.Icon(color =c)).add_to(map_)
    folium.Marker(line[1], popup = 'pt1' , icon = folium.Icon(color =c)).add_to(map_)
    
    folium.PolyLine( line , color=c, weight=2.5, opacity=1).add_to(map_)
    #break

#    folium.PolyLine( line , color="red", weight=2.5, opacity=1).add_to(map_)
folium.Marker(signalCoordinates[idx,:].tolist()[0], popup = 'sign', icon = folium.Icon(color ='blue')).add_to(map_)
#folium.Marker(signalCoordinates[idx,:].tolist()[0], popup = 'sign', icon = folium.Icon(color ='blue')).add_to(map_)
map_.create_map(path='test1.html')
map_


[[ 45.51768059 -73.56166281]] - [[ 0.00019367  0.00017821]]
[[ 0.00026319]] [[ 0.00093571]] [[ 0.00093571]] [[ 0.00034428  0.0003168 ]] [[  8.75554088e-07]]
black


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:34: FutureWarning: Map.create_map is deprecated. Use Map.save instead


In [15]:
0.00039043
0.00031193
dist

matrix([[ 0.00030075]])

In [8]:
import folium
%matplotlib inline
map_ = folium.Map(location=[45.5017, -73.5673], zoom_start=11)

for i in range(10):
    idx = np.random.randint(0, signalCoordinates.shape[0],1)
    dist, pair, pj, pjs, dists, allMats = findClosest( allPairsMat, signalCoordinates[idx,:]  )
    
    line = pair.tolist()[0]
    line = [(line[0], line[1]), (line[2], line[3])]
    print(line, pj, signalCoordinates[idx, :])
    folium.Marker(signalCoordinates[idx,:].tolist()[0], popup = 'sign' + str(idx), icon = folium.Icon(color ='blue')).add_to(map_)
    folium.Marker(pj.tolist()[0], popup = 'pj', icon = folium.Icon(color ='red')).add_to(map_)
    folium.PolyLine( line , color="red", weight=2.5, opacity=1).add_to(map_)
map_


[(45.5239927185675, -73.5487277402396), (45.523752917457, -73.5482345390904)] [[ 45.52399272 -73.54872774]] [[ 45.52392845 -73.5487657 ]]
[(45.5926629837478, -73.6158452805202), (45.5911308340866, -73.6123745690003)] [[ 45.59266296 -73.61584523]] [[ 45.59260047 -73.61584675]]
[(45.5054375284914, -73.6267780705862), (45.5050784826376, -73.6269976610709)] [[ 45.50543753 -73.62677807]] [[ 45.50510691 -73.62669851]]
[(45.501744959804, -73.5763442936734), (45.5018500620878, -73.5764930470302)] [[ 45.50174496 -73.57634429]] [[ 45.50179667 -73.57640294]]
[(45.6554431650241, -73.4936637050601), (45.6554440648489, -73.4936638109445)] [[ 45.65544317 -73.49366371]] [[ 45.6551203  -73.49358803]]
[(45.5986572033295, -73.5156956026289), (45.5982850572846, -73.5159227094761)] [[ 45.5986572 -73.5156956]] [[ 45.59858799 -73.51578408]]
[(45.5203707878549, -73.559713846626), (45.5200644010076, -73.5599843400801)] [[ 45.52037079 -73.55971385]] [[ 45.52059374 -73.55991569]]
[(45.5764871359214, -73.54347183

In [ ]:
allCoordinates[0]